## Code to get the average of the MDE results

In [4]:
import pandas as pd
import os

def extract_satellite_type(filename):
    """
    Extract satellite type from filename.
    Returns 'S1', 'ENV', or 'ERS' based on the prefix.
    """
    if filename.startswith('S1'):
        return 'S1'
    elif filename.startswith('ENV'):
        return 'ENV'
    elif filename.startswith('ERS'):
        return 'ERS'
    else:
        return 'Unknown'

model_path = "/home/users/amorgan/benchmark_CB_AM/mde_results_filtered"
folders = ["DeepLabV3_best_iou", "DinoV3_best_iou", "FPN_best_iou", "Unet_best_iou", "ViT_best_iou"]

all_results = {}

for folder in folders:
    # Construct path to detailed_results.csv
    csv_path = os.path.join(model_path, folder, "detailed_results.csv")
    
    if os.path.exists(csv_path):
        # Read the CSV file
        df = pd.read_csv(csv_path)
        
        # Extract satellite type from filename
        df['satellite_type'] = df['filename'].apply(extract_satellite_type)
        
        # Calculate average distance by satellite type
        avg_by_satellite = df.groupby('satellite_type')['distance_m'].mean()
        
        # Store results
        all_results[folder] = avg_by_satellite.to_dict()
        
        print(f"\n{folder}:")
        for satellite, avg_distance in avg_by_satellite.items():
            print(f"  {satellite}: {avg_distance:.2f} m")
    else:
        print(f"Warning: {csv_path} not found")

# Print summary table
print("\n" + "="*70)
print("SUMMARY TABLE - Average Distance by Satellite Type (meters)")
print("="*70)
print(f"{'Model':<20} {'ENV':<12} {'ERS':<12} {'S1':<12}")
print("-" * 60)

for model, results in all_results.items():
    env_avg = results.get('ENV', 0)
    ers_avg = results.get('ERS', 0)
    s1_avg = results.get('S1', 0)
    print(f"{model:<20} {env_avg:<12.2f} {ers_avg:<12.2f} {s1_avg:<12.2f}")


DeepLabV3_best_iou:
  ENV: 487.00 m
  ERS: 688.88 m
  S1: 590.61 m

DinoV3_best_iou:
  ENV: 595.33 m
  ERS: 865.26 m
  S1: 587.70 m

FPN_best_iou:
  ENV: 508.40 m
  ERS: 423.12 m
  S1: 715.19 m

Unet_best_iou:
  ENV: 559.87 m
  ERS: 620.33 m
  S1: 673.45 m

ViT_best_iou:
  ENV: 589.61 m
  ERS: 736.86 m
  S1: 693.00 m

SUMMARY TABLE - Average Distance by Satellite Type (meters)
Model                ENV          ERS          S1          
------------------------------------------------------------
DeepLabV3_best_iou   487.00       688.88       590.61      
DinoV3_best_iou      595.33       865.26       587.70      
FPN_best_iou         508.40       423.12       715.19      
Unet_best_iou        559.87       620.33       673.45      
ViT_best_iou         589.61       736.86       693.00      


In [2]:
# code for just the mean
for folder in folders:
    # Construct path to detailed_results.csv
    csv_path = os.path.join(model_path, folder, "detailed_results.csv")
    
    if os.path.exists(csv_path):
        # Read the CSV file
        df = pd.read_csv(csv_path)
        
        # Calculate overall mean distance
        overall_mean = df['distance_m'].mean()
        
        print(f"{folder}: {overall_mean:.2f} m")
    else:
        print(f"Warning: {csv_path} not found")

# Print summary table for overall means
print("\n" + "="*40)
print("OVERALL MEAN DISTANCES (meters)")
print("="*40)

for folder in folders:
    csv_path = os.path.join(model_path, folder, "detailed_results.csv")
    
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        overall_mean = df['distance_m'].mean()
        print(f"{folder:<20} {overall_mean:<8.2f}")

DeepLabV3_best_iou: 354.11 m
DinoV3_best_iou: 536.66 m
FPN_best_iou: 530.62 m
Unet_best_iou: 515.24 m
ViT_best_iou: 356.55 m

OVERALL MEAN DISTANCES (meters)
DeepLabV3_best_iou   354.11  
DinoV3_best_iou      536.66  
FPN_best_iou         530.62  
Unet_best_iou        515.24  
ViT_best_iou         356.55  


In [3]:
# Check sample counts by satellite type
print("\n" + "="*70)
print("SAMPLE COUNTS BY SATELLITE TYPE")
print("="*70)
print(f"{'Model':<20} {'ENV':<8} {'ERS':<8} {'S1':<8} {'Total':<8}")
print("-" * 60)

for folder in folders:
    csv_path = os.path.join(model_path, folder, "detailed_results.csv")
    
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        
        # Count samples by satellite type
        counts = df['satellite_type'].value_counts()
        
        env_count = counts.get('ENV', 0)
        ers_count = counts.get('ERS', 0)
        s1_count = counts.get('S1', 0)
        total_count = len(df)
        
        print(f"{folder:<20} {env_count:<8} {ers_count:<8} {s1_count:<8} {total_count:<8}")

# Verify the weighted average calculation for one model
print("\n" + "="*50)
print("WEIGHTED AVERAGE VERIFICATION (DeepLabV3 example)")
print("="*50)

folder = "DeepLabV3_best_iou"
csv_path = os.path.join(model_path, folder, "detailed_results.csv")
df = pd.read_csv(csv_path)

# Get counts and means
counts = df['satellite_type'].value_counts()
means = df.groupby('satellite_type')['distance_m'].mean()

env_count, env_mean = counts.get('ENV', 0), means.get('ENV', 0)
ers_count, ers_mean = counts.get('ERS', 0), means.get('ERS', 0)
s1_count, s1_mean = counts.get('S1', 0), means.get('S1', 0)

# Calculate weighted average manually
weighted_avg = (env_count * env_mean + ers_count * ers_mean + s1_count * s1_mean) / (env_count + ers_count + s1_count)
simple_avg = (env_mean + ers_mean + s1_mean) / 3
overall_mean = df['distance_m'].mean()

print(f"ENV: {env_count} samples × {env_mean:.2f}m = {env_count * env_mean:.2f}")
print(f"ERS: {ers_count} samples × {ers_mean:.2f}m = {ers_count * ers_mean:.2f}")
print(f"S1:  {s1_count} samples × {s1_mean:.2f}m = {s1_count * s1_mean:.2f}")
print(f"Weighted Average: {weighted_avg:.2f}m")
print(f"Simple Average:   {simple_avg:.2f}m")
print(f"Overall Mean:     {overall_mean:.2f}m")



SAMPLE COUNTS BY SATELLITE TYPE
Model                ENV      ERS      S1       Total   
------------------------------------------------------------
DeepLabV3_best_iou   119      426      1216     1761    
DinoV3_best_iou      123      451      1281     1855    
FPN_best_iou         123      414      1164     1701    
Unet_best_iou        121      446      1243     1810    
ViT_best_iou         114      419      1187     1720    

WEIGHTED AVERAGE VERIFICATION (DeepLabV3 example)
ENV: 119 samples × 304.98m = 36292.29
ERS: 426 samples × 426.95m = 181880.80
S1:  1216 samples × 333.40m = 405415.51
Weighted Average: 354.11m
Simple Average:   355.11m
Overall Mean:     354.11m
